## CME DETECTION using csvv files

In [1]:
import pandas as pd

def detect_cme_events(df, 
                      b_threshold=10,         # magnetic field magnitude threshold (nT)
                      density_threshold=10,   # proton density threshold (cm^-3)
                      speed_threshold=500,    # solar wind speed threshold (km/s)
                      window='30T'):          # smoothing window

    """
    Detects potential CME events based on sudden changes in magnetic field, 
    proton density, and solar wind speed.
    
    Parameters:
        df: Pandas DataFrame with datetime index and necessary columns
        b_threshold: Threshold for magnetic field strength in nT
        density_threshold: Threshold for proton density
        speed_threshold: Threshold for solar wind speed
        window: Resampling/smoothing window (e.g., '30T' for 30 minutes)
        
    Returns:
        df_events: DataFrame with rows where a CME is detected
    """

    # Step 1: Smooth (optional)
    df_smoothed = df.resample(window).mean().interpolate()

    # Step 2: Thresholds
    b_detect = df_smoothed["MAG_B"].fillna(0) > b_threshold
    density_detect = df_smoothed["SW_Density"].fillna(0) > density_threshold
    speed_detect = df_smoothed["SW_Speed"].fillna(0) > speed_threshold

    # Step 3: Combined logic (all 3 must trigger)
    cme_mask = b_detect & density_detect & speed_detect

    # Step 4: Return events
    df_events = df_smoothed[cme_mask]

    return df_events



---

## ✅ 1. **Threshold Calculation (Formulaic Approach)**

In most space weather and heliophysics literature, **CME detection thresholds** are based on **statistical anomalies** or **empirical observations**.

A **commonly used formula** to define the threshold is:

$$
\text{Threshold} = \mu + N \cdot \sigma
$$

Where:

* $\mu$ = mean of the signal
* $\sigma$ = standard deviation
* $N$ = multiplier (usually between **2** to **3** depending on sensitivity)

This applies to parameters like:

* Magnetic field magnitude $B_T$
* Solar wind density
* Proton speed
* Temperature
* Plasma beta

This approach detects **outliers**, which often correspond to CME events due to **sudden spikes**.

---

## ✅ 2. **CME Detection Criteria (From Research)**

Several published criteria exist. Here's a simplified version used by NASA and academic research papers:

### 🌪 CME Identification Based on ACE Data:

| Parameter            | Condition / Threshold                                        |
| -------------------- | ------------------------------------------------------------ |
| **Magnetic field**   | $B_T > 10 \, \text{nT}$                                      |
| **Density drop**     | $\text{Density} < 5 \, \text{cm}^{-3}$                       |
| **Speed jump**       | $V_p > 500 \, \text{km/s}$                                   |
| **Temperature drop** | $T_p < 10^5 \, \text{K}$                                     |
| **Plasma beta**      | $\beta < 0.5$ (low plasma beta indicates magnetic dominance) |

> These conditions **don’t all have to be met**—commonly, a CME is flagged if **2 or more** criteria are met **over a short duration** (like 1–3 hours).

---


In [2]:
import pandas as pd
import numpy as np
class CMEDetection:
    def __init__(self):
        pass

    def load_file(self, file_path):
        return pd.read_csv(file_path)
        


In [3]:
class_detect = CMEDetection()

In [4]:
epam_data = class_detect.load_file('csv_file/ace_epam_data.csv')

C:\Users\harsh\AppData\Local\Temp\ipykernel_1840\1145778639.py:8: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [5]:
mag_data = class_detect.load_file('csv_file/ace_mag_data.csv')

C:\Users\harsh\AppData\Local\Temp\ipykernel_1840\1145778639.py:8: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [6]:
swepam_data = class_detect.load_file('csv_file/ace_swepam_data.csv')

C:\Users\harsh\AppData\Local\Temp\ipykernel_1840\1145778639.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [7]:
df_epam = epam_data.head(1000)
df_mag = mag_data.head(1000)
df_swics = swepam_data.head(1000)



In [8]:
def add_timestamp_column(df):
    # Convert HHMM (e.g., 1230 → "12:30") safely
    df = df.copy()  # avoid SettingWithCopyWarning

    # Ensure 4-digit zero-padded strings for HHMM
    df["HHMM"] = df["HHMM"].astype(str).str.zfill(4)

    # Construct timestamp string like "2023-07-04 12:30"
    datetime_str = (
        df["YR"].astype(str) + "-" +
        df["MO"].astype(str).str.zfill(2) + "-" +
        df["DA"].astype(str).str.zfill(2) + " " +
        df["HHMM"].str[:2] + ":" + df["HHMM"].str[2:]
    )

    # Parse with known format
    df["timestamp"] = pd.to_datetime(datetime_str, format="%Y-%m-%d %H:%M", errors="coerce")
    df.set_index("timestamp", inplace=True)
    
    return df


In [9]:
df_epam = add_timestamp_column(df_epam)
df_mag = add_timestamp_column(df_mag)
df_swics = add_timestamp_column(df_swics)


In [10]:
df_epam, df_mag, df_swics

(               YR   MO    DA    HHMM  Julian_Day  Seconds_of_Day  Electron_S  \
 timestamp                                                                      
 NaT        2001.0  8.0   7.0    00.0     52128.0             0.0         0.0   
 NaT        2001.0  8.0   7.0    05.0     52128.0           300.0         0.0   
 NaT        2001.0  8.0   7.0    10.0     52128.0           600.0         0.0   
 NaT        2001.0  8.0   7.0    15.0     52128.0           900.0         0.0   
 NaT        2001.0  8.0   7.0    20.0     52128.0          1200.0         0.0   
 ...           ...  ...   ...     ...         ...             ...         ...   
 NaT        2001.0  8.0  10.0  1055.0     52131.0         39300.0         0.0   
 NaT        2001.0  8.0  10.0  1100.0     52131.0         39600.0         0.0   
 NaT        2001.0  8.0  10.0  1105.0     52131.0         39900.0         0.0   
 NaT        2001.0  8.0  10.0  1110.0     52131.0         40200.0         0.0   
 NaT        2001.0  8.0  10.

In [11]:
def compute_thresholds(df, columns, k=2):
    thresholds = {}
    for col in columns:
        if df[col].dropna().empty:
            thresholds[col] = None
        else:
            mean = df[col].mean()
            std = df[col].std()
            thresholds[col] = mean + k * std
    return thresholds


In [12]:
df_epam.columns

Index(['YR', 'MO', 'DA', 'HHMM', 'Julian_Day', 'Seconds_of_Day', 'Electron_S',
       'Electron_38-53', 'Electron_175-315', 'Proton_S', 'Proton_47-65',
       'Proton_112-187', 'Proton_310-580', 'Proton_761-1220',
       'Proton_1060-1910', 'Anisotropy_Ratio', 'year', 'file'],
      dtype='object')

In [13]:
df_swics.columns

Index(['YR', 'MO', 'DA', 'HHMM', 'Julian_Day', 'Seconds_of_Day', 'S',
       'Proton_Density', 'Bulk_Speed', 'Ion_Temperature', 'year', 'file'],
      dtype='object')

In [14]:
df_mag.columns

Index(['YR', 'MO', 'DA', 'HHMM', 'Julian_Day', 'Seconds_of_Day', 'S', 'Bx',
       'By', 'Bz', 'Bt', 'Latitude', 'Longitude', 'year', 'file'],
      dtype='object')

In [15]:
sis_data = class_detect.load_file('csv_file/ace_sis_data.csv')

C:\Users\harsh\AppData\Local\Temp\ipykernel_1840\1145778639.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [16]:
df_sis = sis_data.head(1000)

In [17]:
df_sis.columns

Index(['YR', 'MO', 'DA', 'HHMM', 'Julian_Day', 'Seconds_of_Day', 'S_10MeV',
       'Proton_>10MeV', 'S_30MeV', 'Proton_>30MeV', 'year', 'file'],
      dtype='object')

In [18]:
thresholds_epam = compute_thresholds(df_sis, ["Proton_>10MeV", "Proton_>30MeV"], k=3)
thresholds_mag = compute_thresholds(df_mag, ["Bz", "Bt"], k=2)
thresholds_swics = compute_thresholds(df_swics, ["Proton_Density", "Bulk_Speed"], k=2)


In [19]:
df_sis["sep_event"] = (
    (df_sis["Proton_>10MeV"] > thresholds_epam["Proton_>10MeV"]) |
    (df_sis["Proton_>30MeV"] > thresholds_epam["Proton_>30MeV"])
)

df_mag["mag_event"] = df_mag["Bz"] < (df_mag["Bz"].mean() - 2 * df_mag["Bz"].std())

df_swics["shock_event"] = (
    (df_swics["Proton_Density"] > thresholds_swics["Proton_Density"]) |
    (df_swics["Bulk_Speed"] > thresholds_swics["Bulk_Speed"])
)


C:\Users\harsh\AppData\Local\Temp\ipykernel_1840\153231050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sis["sep_event"] = (


In [20]:
df_sis['sep_event'].value_counts()

sep_event
False    996
True       4
Name: count, dtype: int64

In [21]:
# Ensure consistent datetime indexes
df_sis.index = pd.to_datetime(df_sis.index)
df_mag.index = pd.to_datetime(df_mag.index)
df_swics.index = pd.to_datetime(df_swics.index)

# Now do the join
df_combined = (
    df_sis[["sep_event"]]
    .join(df_mag[["mag_event"]], how="outer")
    .join(df_swics[["shock_event"]], how="outer")
    .fillna(False)
)

In [22]:
df_combined.value_counts()

sep_event  mag_event  shock_event
False      False      False          2933
                      True             63
True       False      False             4
Name: count, dtype: int64

In [23]:
df_sis

,YR,MO,DA,HHMM,Julian_Day,Seconds_of_Day,S_10MeV,Proton_>10MeV,S_30MeV,Proton_>30MeV,year,file,sep_event
1970-01-01 00:00:00.000000000,2001.0,8.0,7.0,0.0,52128.0,0.0,0.0,0.797,0.0,0.557,2001,20010807_ace_sis_5m.txt,False
1970-01-01 00:00:00.000000001,2001.0,8.0,7.0,5.0,52128.0,300.0,0.0,0.800,0.0,0.567,2001,20010807_ace_sis_5m.txt,False
1970-01-01 00:00:00.000000002,2001.0,8.0,7.0,10.0,52128.0,600.0,0.0,0.796,0.0,0.564,2001,20010807_ace_sis_5m.txt,False
1970-01-01 00:00:00.000000003,2001.0,8.0,7.0,15.0,52128.0,900.0,0.0,0.803,0.0,0.563,2001,20010807_ace_sis_5m.txt,False
1970-01-01 00:00:00.000000004,2001.0,8.0,7.0,20.0,52128.0,1200.0,0.0,0.831,0.0,0.587,2001,20010807_ace_sis_5m.txt,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:00:00.000000995,2001.0,8.0,10.0,1055.0,52131.0,39300.0,0.0,25.600,0.0,1.160,2001,20010810_ace_sis_5m.txt,False
1970-01-01 00:00:00.000000996,2001.0,8.0,10.0,1100.0,52131.0,39600.0,0.0,28.200,0.0,1.290,2001,20010810_ace_sis_5m.txt,False
1970-01-01 00:00:00.000000997,2001.0,8.0,10.0,1105.0,52131.0,39900.0,0.0,28.200,0.0,1.180,2001,20010810_ace_sis_5m.txt,False
1970-01-01 00:00:00.000000998,2001.0,8.0,10.0,1110.0,52131.0,40200.0,0.0,24.000,0.0,0.938,2001,20010810_ace_sis_5m.txt,False


In [24]:
df_sis.set_index("timestamp", inplace=True)
df_mag.set_index("timestamp", inplace=True)
df_swics.set_index("timestamp", inplace=True)

KeyError: "None of ['timestamp'] are in the columns"

In [25]:
df_combined = df_sis[["sep_event"]].join(df_mag[["mag_event"]], how="outer").join(df_swics[["shock_event"]], how="outer").fillna(False)

# Final CME condition
df_combined["cme_detected"] = (
    df_combined["sep_event"] & df_combined["mag_event"] & df_combined["shock_event"]
)


In [27]:
df_combined["event_group"] = (df_combined["cme_detected"] != df_combined["cme_detected"].shift()).cumsum()
cme_events = df_combined[df_combined["cme_detected"]].groupby("event_group").agg({"cme_detected": ["count", "idxmin", "idxmax"]})
cme_events.columns = ["duration", "start_time", "end_time"]


In [28]:
print("Detected CME Events:")
print(cme_events[["start_time", "end_time", "duration"]])


Detected CME Events:
Empty DataFrame
Columns: [start_time, end_time, duration]
Index: []


In [29]:
cme_events

,duration,start_time,end_time
event_group,,,


In [ ]:
from src.threshold_optimization import *
from src.cme_detection_system import CMEDetectionSystem

cme_files = CMEDetectionSystem('csv_file')

NameError: name 'CMEDetectionSystem' is not defined